In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import torch.nn as nn
import torch
import math

from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings(action='ignore')

import sys
sys.path.append('/root/daily/bit')

from Prescriptor import Prescriptor
from Evolution.crossover import UniformCrossover, WeightedSumCrossover, SkipCrossover, DifferentialEvolutionOperator
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation, RandomValueMutation
from Evolution.selection import RouletteSelection, TournamentSelection
from Evolution import Evolution

In [2]:
from typing import Any, Callable, List, Union
from numbers import Integral, Real
from copy import deepcopy

import torch
import numpy as np

from tqdm import tqdm

class Evolution:
    def __init__(self, prescriptor, selection, crossover, mutation, group_size):
        self.prescriptor = prescriptor
        self.selection = selection
        self.crossover = crossover
        self.mutation = mutation
        
        self.chromosome_size = group_size
        self.num_parents = self.crossover.get_num_parents()

        self.check_model_shape()

    # def check_model_shape(self):
    #     device = next(self.prescriptor.parameters()).device
    #     self.prescriptor.cpu()
    #     self.shape_each_layer = []
    #     self.num_each_layer = []
    #     for name, param in self.prescriptor.layers[0].named_parameters():
    #         size = list(param.size())
    #         self.shape_each_layer.append(size)
    #         layer_param = 1
    #         for idx, item in enumerate(size):
    #             layer_param *= item
    #         self.num_each_layer.append(layer_param)

    #     self.prescriptor = self.prescriptor.to(device)
    
    def check_model_shape(self):
        device = next(self.prescriptor.parameters()).device
        self.prescriptor.cpu()
        # Base layers
        self.shape_each_layer = []
        self.num_each_layer = []
        for name, param in self.prescriptor.base_layers.named_parameters():
            size = list(param.size())
            self.shape_each_layer.append(size)
            self.num_each_layer.append(param.numel())

        # After layers
        self.after_shape_each_layer = []
        self.after_num_each_layer = []
        for name, param in self.prescriptor.after_layers.named_parameters():
            size = list(param.size())
            self.after_shape_each_layer.append(size)
            self.after_num_each_layer.append(param.numel())

        self.prescriptor = self.prescriptor.to(device)
    
    def update_chromosomes(self, chromosomes, base_shape, after_shape, device='cpu'):
        chromosomes_size = len(chromosomes)
        base_chromosomes = chromosomes[:, :base_shape[1]]
        after_chromosomes = chromosomes[:, base_shape[1]:]
        with torch.no_grad():
            # Update base layers
            
            sd = self.prescriptor.base_layers.state_dict()
            split_base = 0
            for idx_sd, param_name in enumerate(sd):
                split_margin = split_base + self.num_each_layer[idx_sd] // chromosomes_size
                param = base_chromosomes[:, split_base:split_margin].reshape(self.shape_each_layer[idx_sd])
                sd[param_name] = param
                split_base = split_margin
            self.prescriptor.base_layers.load_state_dict(sd)

            sd = self.prescriptor.after_layers.state_dict()
            split_base = 0
            for idx_sd, param_name in enumerate(sd):
                split_margin = split_base + self.after_num_each_layer[idx_sd] // chromosomes_size
                param = after_chromosomes[:, split_base:split_margin].reshape(self.after_shape_each_layer[idx_sd])
                sd[param_name] = param
                split_base = split_margin
            self.prescriptor.after_layers.load_state_dict(sd)
        self.prescriptor.to(device)


    def flatten_chromosomes(self):
        base_chromosomes, device = self.base_flatten_chromosomes()
        after_chromosomes, _ = self.after_flatten_chromosomes()

        base_ch_shape = base_chromosomes.shape
        after_ch_shape = after_chromosomes.shape
        chromosomes = torch.cat([base_chromosomes, after_chromosomes], dim=1)
        return chromosomes.cpu(), base_ch_shape, after_ch_shape, device

    def base_flatten_chromosomes(self,):
        device = next(self.prescriptor.parameters()).device
        chromosomes_size = self.prescriptor.num_blcoks
        self.prescriptor.cpu()
        with torch.no_grad():
            chromosomes = []
            ch = self.prescriptor.base_layers
            for name, param in ch.named_parameters():
                param = param.flatten().reshape(chromosomes_size, -1)
                chromosomes.append(param)
            chromosomes = torch.concat(chromosomes, dim=1)
        return chromosomes, device

    
    def after_flatten_chromosomes(self, ):
        device = next(self.prescriptor.parameters()).device
        chromosomes_size = self.prescriptor.num_blcoks
        self.prescriptor.cpu()
        with torch.no_grad():
            chromosomes = []
            ch = self.prescriptor.after_layers
            for name, param in ch.named_parameters():
                param = param.flatten().reshape(chromosomes_size, -1)
                chromosomes.append(param)
            chromosomes = torch.concat(chromosomes, dim=1)
        return chromosomes, device
    
    def verify_base_parameters(self):
        original_params = []
        for param in self.prescriptor.base_layers.parameters():
            original_params.append(param.clone())

        # Flatten and reload chromosomes
        chromosomes, base_ch_shape, after_ch_shape, device = self.flatten_chromosomes()
        self.update_chromosomes(chromosomes, base_ch_shape, after_ch_shape, device)

        # Check if parameters are the same
        for original_param, param in zip(original_params, self.prescriptor.base_layers.parameters()):
            if not torch.allclose(original_param, param):
                print("Parameters do not match base reload.")
                return False
        print("Parameters match base reload.")
        return True

    def verify_after_parameters(self):
        original_params = []
        for param in self.prescriptor.after_layers.parameters():
            original_params.append(param.clone())

        # Flatten and reload chromosomes
        chromosomes, base_ch_shape, after_ch_shape, device = self.flatten_chromosomes()
        self.update_chromosomes(chromosomes, base_ch_shape, after_ch_shape, device)

        # Check if parameters are the same
        for original_param, param in zip(original_params, self.prescriptor.after_layers.parameters()):
            if not torch.allclose(original_param, param):
                print("Parameters do not match after reload.")
                return False
        print("Parameters match after reload.")
        return True


    
    def select_elite(self, fitness: torch.Tensor, chromosomes: torch.Tensor, num_elite_chromosomes: int):
        self.selection.select(fitness)
        elite_idx = self.selection.sort_idx()[:num_elite_chromosomes] # for single
        # elite_idx = self.selection.sort_idx(fitness, num_elite_chromosomes).long() # for multi
        elite_chromosomes = chromosomes[elite_idx]
                
        return elite_idx, elite_chromosomes


    def evolve(self, fitness: torch.Tensor):
        # chromosomes = self.prescriptor.chromosomes.cpu()
        chromosomes, base_ch_shape, after_ch_shape, device = self.flatten_chromosomes()
        
        self.selection.select(fitness)
        elite_idx = self.selection.elite_idx()
        elite_chromosomes = deepcopy(chromosomes[elite_idx])
        offspring_size = self.chromosome_size - len(elite_idx)
        select_parents_idx = self.selection.pick_parents(self.num_parents, offspring_size)
        
        select_parents_idx = select_parents_idx.T.flatten()
        parents = chromosomes[select_parents_idx].reshape(offspring_size, 4, -1)

        
        offspring = self.crossover(parents)
        offspring = self.mutation(offspring)

        # print(offspring.shape)

        chromosomes = torch.concat([elite_chromosomes, offspring])
        chromosomes = chromosomes.squeeze(dim=0)
        
        self.update_chromosomes(chromosomes, base_ch_shape, after_ch_shape, device)



In [3]:
device = 'cuda:0'
group = 30

base_x = torch.randn(size=(1, 8)).to(device).float()
base_x = torch.concat([base_x for i in range(group)])
cate_x = torch.randint(0, 3, size=(group, )).to(device)
step = torch.arange(0, group*3, step=3).to(device)

import pickle

# Load data from the file into separate variables
with open('/root/daily/bit_5/backup_feature_data/data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

dataset_1m = loaded_data['dataset_1m']
# dataset_1d = loaded_data['dataset_1d']
skip_data_cnt = loaded_data['skip_data_cnt']
entry_pos_list = loaded_data['entry_pos_list']
bb_macd_entry_pos_list = loaded_data['bb_macd_entry_pos_list']
bb_macd_entry_index_list = loaded_data['bb_macd_entry_index_list']

In [4]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

prescriptor = Prescriptor(input_dim=54, 
                    fc_hidden_size=16, 
                    output_dim=8, 
                    after_input_dim=11, 
                    after_hidden_dim=16, 
                    after_output_dim=5, 
                    num_blocks=group).to(device).eval()
selection = RouletteSelection(elite_num=10, parents_num=62000, minimize=False)
# selection = ParetoLexsortSelection(elite_num=2000, parents_num=4000,
#                                     priority=[], prior_ratio= [],
#                                     prob_method= 'softmax',minimize=False)
# crossover = DifferentialEvolutionOperator()
# crossover = UniformCrossover(num_parents=4)
# crossover = CenDE_DOBLOperator()
mutation = ChainMutation([RandomValueMutation(mut_prob=0.05), AddUniformMutation(mut_prob=0.1)])
# crossover = UniformCrossover(num_parents=2)
crossover = DifferentialEvolutionOperator()
# mutation = AddNormalMutation(mut_prob=0.1)
evolution = Evolution(
    prescriptor=prescriptor,
    selection=selection,
    crossover=crossover,
    mutation=mutation,
    group_size=group
)
get_n_params(prescriptor)

197370

In [22]:
# Flatten and reload chromosomes
chromosomes_1, base_ch_shape, after_ch_shape, device = evolution.flatten_chromosomes()
evolution.update_chromosomes(chromosomes_1, base_ch_shape, after_ch_shape, device)
chromosomes_2, base_ch_shape, after_ch_shape, device = evolution.flatten_chromosomes()

In [ ]:

# chromosomes, base_ch_shape, after_ch_shape, device = evolution.flatten_chromosomes()
# evolution.update_chromosomes(chromosomes, base_ch_shape, after_ch_shape, device)

In [ ]:
with torch.no_grad():
    for i in range(34):
        lstm_logit_2 = pres.base_forward(small_lstm_x, large_lstm_x).squeeze(dim=2)
        # lstm_logit = torch.concat(lstm_logit, dim=1).to(device)
        break
lstm_logit_2 = lstm_logit_2.permute([1, 0, 2])

In [ ]:
pres.eval()
pres.to(device)
for i in range(17000):
    with torch.no_grad():
        after_input_2 = torch.concat([lstm_logit_2[0].cuda(), base_x.cuda()], dim=1)
        after_output_2 = pres.after_forward(x=after_input_2, x_cate=cate_x+step)
        break